In [1]:
import numpy as np
import random
import pandas as pd


In [2]:
istanbul_metro = {
    "M2": [
        "YENIKAPI", "VEZNECILER", "HALIC", "SISHANE", "TAKSIM",
        "OSMANBEY", "SISLI", "GAYRETTEPE", "LEVENT", "4 LEVENT",
        "SANAYI MAH.", "SEYRANTEPE", "ITU", "ATATURK OTO SANAYI", "DARUSSAFAKA",
        "HACIOSMAN"
    ],
    "M3": [
        "BAKIRKOY IDO", "OZGURLUK MEYDANI", "INCIRLI", "HAZNEDAR", "ILKYUVA",
        "MOLLA GURANI", "KIRAZLI", "YENI MAHALLE", "MAHMUTBEY", "ISTOC",
        "IKITELLI SANAYI", "TURGUT OZAL", "SITELER", "BASAK KONUTLARI", "METROKENT",
        "ONURKENT", "SEHIR HASTANESI", "TOPLU KONUTLAR", "KAYASEHIR MERKEZ"
    ],
    "M4": [
        "KADIKOY", "AYRILIKCESME", "ACIBADEM", "UNALAN", "GOZTEPE",
        "YENISAHRA", "KOZYATAGI", "BOSTANCI", "KUCUKYALI", "MALTEPE",
        "HUZUREVI", "GULSUYU", "ESENKENT CEVIZLI", "HASTANE", "SOGANLIK",
        "KARTAL", "YAKACIK", "PENDIK", "TAVSANTEPE", "FEVZI CAKMAK",
        "YAYALAR", "KURTKOY", "SABIHA GOKCEN HAVALIMANI",
    ],
    "M5": [
        "USKUDAR", "FISTIKAGACI", "BAGLARBASI", "ALTUNIZADE", "KISIKLI",
        "BULGURLU", "UMRANIYE", "CARSI", "YAMANEVLER", "CAKMAK",
        "IHLAMURKUYU", "ALTINSEHIR", "IMAM HATIP LISESI", "DUDULLU", "NECIP FAZIL",
        "CEKMEKOY", "MECLIS", "SARIGAZI", "SANCAKTEPE", "SAMANDIRA MERKEZ"
    ],
    "M6": [
        "LEVENT  KONKORS", "NISPETIYE", "ETILER", "BOGAZICI"
    ],
    "M7": [
        "YILDIZ", "FULYA", "MECIDIYEKOY", "CAGLAYAN", "KAGITHANE",
        "NURTEPE", "ALIBEYKOY", "CIRCIR", "VEYSELKARANI", "YESILPINAR",
        "KAZIMKARABEKIR", "YENIMAHALLE", "KARADENIZ MAH.", "TEKSTILKENT", "ORUCREIS",
        "GOZTEPE", "MAHMUTBEY M3 HOL"
    ],
    "M8": [
        "BOSTANCI", "EMIN ALI PASA", "AYSE KADIN", "KOZYATAGI", "KUCUKBAKKALKOY",
        "ICERENKOY", "KAYISDAGI", "MEVLANA", "IMES", "MODOKO KEYAP", 
        "DUDULLU", "HUZUR", "PARSELLER"
    ],
    "M9": [
        "ATAKOY", "YENIBOSNA", "COBANCESME", "29 EKIM CUMHURIYETIHLAS YUVA", "SANAYI", "MIMAR SINAN",
        "15 TEMMUZ", "HALKALI", "ATATURK MAHALLESI", "BAHARIYE", "MASKO", 
        "IKITELLI SANAYI", "ZIYA GOKALP MAH", "OLIMPIYAT"
    ],

    "T5": [
        "EMINONU", "KUCUKPAZAR", "CIBALI", "FENER", "BALAT",
        "AYVANSARAY", "FESHANE", "EYUP TELEFERIK", "EYUP DEVLET HASTANESI", "SILAHTARAGA",
        "UNIVERSITE", "ALIBEYKOY", "ALIBEYKOY METRO", "CEP OTOGAR"],

    "T4": [
        "TOPKAPI", "FETIHKAPI", "VATAN", "EDIRNEKAPI", "SEHITLIK",
        "DEMIRKAPI", "TOPCULAR", "RAMI", "BEREC", "SAGMALCILAR",
        "CUKURCESME", "ALI FUAT BASGIL", "TASKOPRU", "KARADENIZ MAHALLESI", "T4 KIPTAS VENEZIA",
        "CUMHURIYET", "BASTABYA", "HACI SUKRU", "YENIMAHALLE", "SULTANCIFTLIGI",
        "CEBECI", "MESCIDI SELAM"],

    "T1": [ 
        "KABATAS", "FINDIKLI", "TOPHANE", "KARAKOY", "EMINONU",
        "SIRKECI", "GULHANE", "SULTANAHMET", "CEMBERLITAS", "BEYAZIT",
        "LALELI", "AKSARAY", "YUSUFPASA", "HASEKI", "FINDIKZADE",
        "CAPA", "PAZARTEKKE", "TOPKAPI", "ATATURK OGRENCI YURDU", "MERKEZEFENDI",
        "AKSEMSETTIN", "MITHATPASA", "ZEYTINBURNU", "MEHMETAKIF", "...",
        "GUNGOREN", "AKINCILAR", "SOGANLI", "...", "GUNESTEPE",
        "BAGCILAR"]
}


In [3]:
data = pd.read_csv('data/hourly_transportation_202409.csv')
data = data[data["line_name"].isin(["M2", "M3", "M4", "M5", "M6", "M7", "M8", "M9", "T1", "T4", "T5"])]
data = data[["transition_date", "transition_hour", "line_name", "number_of_passage", "station_poi_desc_cd"]]
data = data.rename(columns={"transition_date": "date", "transition_hour": "hour", "line_name": "line", "number_of_passage": "passenger_count", "station_poi_desc_cd": "station"})
data = data[data["hour"] >= 6]

# Remove unwanted words (KUZEY, GUNEY, DOGU, BATI) and numbers from the 'station' column
data['station'] = data['station'].str.replace(r'\b(KUZEY|GUNEY|DOGU|BATI)\b', '', regex=True)
#remove all digits and numbers at the end of the string
data['station'] = data['station'].str.replace(r'\b(T4|M7|M9)\b', '', regex=True)
data['station'] = data['station'].str.replace(r'\s*\d+\s*$', '', regex=True)
data['station'] = data['station'].str.replace(r'[()\-/]', '', regex=True)  # Remove symbols: (), -, /
data['station'] = data['station'].str.strip()
data["station"] = data["station"].astype(str)


data = data.groupby(['date', 'hour', 'line', 'station'], as_index=False)['passenger_count'].sum()
data = data.groupby(['hour', 'line', 'station'], as_index=False).agg({'passenger_count': 'mean'})



data


,hour,line,station,passenger_count
0,6,M2,4 LEVENT,546.125
1,6,M2,ATATURK OTO SANAYI,52.750
2,6,M2,DARUSSAFAKA,62.625
3,6,M2,GAYRETTEPE,328.375
4,6,M2,HACIOSMAN,294.250
...,...,...,...,...
3541,23,T5,FENER,47.875
3542,23,T5,FESHANE,52.750
3543,23,T5,KUCUKPAZAR,106.250
3544,23,T5,SILAHTARAGA,21.875


In [4]:
from collections import OrderedDict

def sort_passenger_data_by_station(data, station_order):
    sorted_data = {}

    for line, hours in data.items():
        if line not in station_order:
            print(f"[UYARI] '{line}' hattı için durak sırası tanımlı değil.")
            continue

        line_order = station_order[line]
        all_stations_in_data = set()
        for hour_data in hours.values():
            all_stations_in_data.update(hour_data.keys())

        # 1. station_order'da olup dict'te olmayanlar
        for station in line_order:
            if station not in all_stations_in_data:
                print(f"[Eksik Veri] '{line}' hattında sıralamada olan '{station}' istasyonu veride yok.")

        # 2. dict'te olup station_order'da olmayanlar
        for station in all_stations_in_data:
            if station not in line_order:
                print(f"[Sıralama Bilgisi Yok] '{line}' hattında veride olan '{station}' istasyonu sıralama listesinde yok.")

        # Asıl sıralama işlemi
        sorted_data[line] = {}
        for hour, stations in hours.items():
            sorted_stations = OrderedDict()
            for station in line_order:
                if station in stations:
                    sorted_stations[station] = stations[station]
            sorted_data[line][hour] = sorted_stations

    return sorted_data


In [5]:
dict = {}
for i in range(len(data)):
    if data["line"][i] not in dict:
        dict[data["line"][i]] = {}
    if data["hour"][i] not in dict[data["line"][i]]:
        dict[data["line"][i]][data["hour"][i]] = {}
    if data["station"][i] not in dict[data["line"][i]][data["hour"][i]]:
        dict[data["line"][i]][data["hour"][i]][data["station"][i]] = {}
    dict[data["line"][i]][data["hour"][i]][data["station"][i]] = data["passenger_count"][i]
dict

{'M2': {6: {'4 LEVENT': 546.125,
   'ATATURK OTO SANAYI': 52.75,
   'DARUSSAFAKA': 62.625,
   'GAYRETTEPE': 328.375,
   'HACIOSMAN': 294.25,
   'HALIC': 96.875,
   'ITU': 100.875,
   'LEVENT': 480.625,
   'OSMANBEY': 343.25,
   'SANAYI MAH.': 139.625,
   'SEYRANTEPE': 104.125,
   'SEYRANTEPE 3 STAD GIRISI': 6.75,
   'SISHANE': 209.375,
   'SISLI': 1175.0,
   'TAKSIM': 347.5,
   'VEZNECILER': 99.0,
   'YENIKAPI': 1198.25},
  7: {'4 LEVENT': 1417.375,
   'ATATURK OTO SANAYI': 181.125,
   'DARUSSAFAKA': 188.375,
   'GAYRETTEPE': 1345.375,
   'HACIOSMAN': 1005.75,
   'HALIC': 236.75,
   'ITU': 371.0,
   'LEVENT': 1013.0,
   'OSMANBEY': 1194.75,
   'SANAYI MAH.': 433.0,
   'SEYRANTEPE': 265.0,
   'SEYRANTEPE 3 STAD GIRISI': 31.125,
   'SISHANE': 528.5,
   'SISLI': 3618.25,
   'TAKSIM': 650.0,
   'VEZNECILER': 254.75,
   'YENIKAPI': 4045.375},
  8: {'4 LEVENT': 1977.0,
   'ATATURK OTO SANAYI': 251.75,
   'DARUSSAFAKA': 309.5,
   'GAYRETTEPE': 2501.625,
   'HACIOSMAN': 1377.625,
   'HALIC': 3

In [6]:
sorted_data = sort_passenger_data_by_station(dict, istanbul_metro)

[Sıralama Bilgisi Yok] 'M2' hattında veride olan 'SEYRANTEPE 3 STAD GIRISI' istasyonu sıralama listesinde yok.
[Sıralama Bilgisi Yok] 'M3' hattında veride olan 'MAHMUTBEY  HOL' istasyonu sıralama listesinde yok.
[Sıralama Bilgisi Yok] 'M4' hattında veride olan 'HASTANE ADLIYE' istasyonu sıralama listesinde yok.
[Sıralama Bilgisi Yok] 'M7' hattında veride olan 'MAHMUTBEY  HOL' istasyonu sıralama listesinde yok.
[Eksik Veri] 'T1' hattında sıralamada olan '...' istasyonu veride yok.
[Eksik Veri] 'T1' hattında sıralamada olan '...' istasyonu veride yok.
[Sıralama Bilgisi Yok] 'T1' hattında veride olan 'KERESTECILER' istasyonu sıralama listesinde yok.
[Sıralama Bilgisi Yok] 'T1' hattında veride olan 'CAMI' istasyonu sıralama listesinde yok.
[Eksik Veri] 'T4' hattında sıralamada olan 'T4 KIPTAS VENEZIA' istasyonu veride yok.
[Sıralama Bilgisi Yok] 'T4' hattında veride olan 'KIPTAS VENEZIA' istasyonu sıralama listesinde yok.


In [7]:
df_rows = []
for line, hours in sorted_data.items():
    for hour, stations in hours.items():
        for station, count in stations.items():
            df_rows.append({
                "line": line, "hour": hour, "station": station, "passenger_count": count
            })

data_sorted = pd.DataFrame(df_rows)
data_sorted

,line,hour,station,passenger_count
0,M2,6,YENIKAPI,1198.250
1,M2,6,VEZNECILER,99.000
2,M2,6,HALIC,96.875
3,M2,6,SISHANE,209.375
4,M2,6,TAKSIM,347.500
...,...,...,...,...
3415,T5,23,SILAHTARAGA,21.875
3416,T5,23,UNIVERSITE,14.250
3417,T5,23,ALIBEYKOY,42.000
3418,T5,23,ALIBEYKOY METRO,129.125


In [8]:
#cumulative sum of passenger_count shifted by 1 station
data_sorted['cum_passenger_count'] = data_sorted.groupby(['hour', 'line'])['passenger_count'].cumsum()
data_sorted['cum_passenger_count'] = data_sorted.groupby(['hour', 'line'])['cum_passenger_count'].shift(fill_value=0)
data_sorted

,line,hour,station,passenger_count,cum_passenger_count
0,M2,6,YENIKAPI,1198.250,0.000
1,M2,6,VEZNECILER,99.000,1198.250
2,M2,6,HALIC,96.875,1297.250
3,M2,6,SISHANE,209.375,1394.125
4,M2,6,TAKSIM,347.500,1603.500
...,...,...,...,...,...
3415,T5,23,SILAHTARAGA,21.875,743.750
3416,T5,23,UNIVERSITE,14.250,765.625
3417,T5,23,ALIBEYKOY,42.000,779.875
3418,T5,23,ALIBEYKOY METRO,129.125,821.875


In [9]:
def calculate_passenger_count(data, hour, line, station):
    data = data[(data["hour"] == hour) & (data["line"] == line)]
    station_data = data[data["station"] == station]
    passenger_count = station_data["passenger_count"].iloc[0]
    total_passenger_count = sum(data["passenger_count"])
    cumsum_passenger_count = station_data["cum_passenger_count"].iloc[0]
    direction_2 = passenger_count * cumsum_passenger_count / (total_passenger_count-passenger_count)
    direction_1 = passenger_count-direction_2
    return direction_1, direction_2

In [10]:
data_sorted['direction_1'] = None
data_sorted['direction_2'] = None

for index, row in data_sorted.iterrows():
    direction_1, direction_2 = calculate_passenger_count(data_sorted, row['hour'], row['line'], row['station'])
    data_sorted.at[index, 'direction_1'] = direction_1
    data_sorted.at[index, 'direction_2'] = direction_2
data_sorted['direction_1'] = pd.to_numeric(data_sorted['direction_1'], errors='coerce')
data_sorted['direction_2'] = pd.to_numeric(data_sorted['direction_2'], errors='coerce')
data_sorted

,line,hour,station,passenger_count,cum_passenger_count,direction_1,direction_2
0,M2,6,YENIKAPI,1198.250,0.000,1198.250000,0.000000
1,M2,6,VEZNECILER,99.000,1198.250,77.351301,21.648699
2,M2,6,HALIC,96.875,1297.250,73.949640,22.925360
3,M2,6,SISHANE,209.375,1394.125,155.010811,54.364189
4,M2,6,TAKSIM,347.500,1603.500,240.980609,106.519391
...,...,...,...,...,...,...,...
3415,T5,23,SILAHTARAGA,21.875,743.750,5.104449,16.770551
3416,T5,23,UNIVERSITE,14.250,765.625,3.091569,11.158431
3417,T5,23,ALIBEYKOY,42.000,779.875,7.521316,34.478684
3418,T5,23,ALIBEYKOY METRO,129.125,821.875,6.135448,122.989552


In [11]:
import pandas as pd

df_orig = data_sorted.copy()  # orijinal DataFrame

new_rows = []

for line in df_orig['line'].unique():
    df_line = df_orig[df_orig['line'] == line].copy()

    for hour, group in df_line.groupby('hour'):
        # Sıralı hale getir
        group_sorted = group.sort_values(by='station', key=lambda s: s.factorize()[0])

        # Orijinal yön (direction_1)
        df_orig_dir = group_sorted.copy()
        df_orig_dir['in_passenger'] = df_orig_dir['direction_1']
        df_orig_dir = df_orig_dir.drop(columns=['direction_2'])

        # Ters yön (direction_2)
        df_rev_dir = group_sorted.iloc[::-1].copy()
        df_rev_dir['line'] = f"{line}-R"
        df_rev_dir['in_passenger'] = df_rev_dir['direction_2']
        df_rev_dir = df_rev_dir.drop(columns=['direction_1'])


        new_rows.extend([df_orig_dir, df_rev_dir])
         


df_split = pd.concat(new_rows, ignore_index=True)
df_split = df_split.drop(columns=["direction_1", "direction_2", "cum_passenger_count", "passenger_count"])
df_split

,line,hour,station,in_passenger
0,M2,6,YENIKAPI,1198.250000
1,M2,6,VEZNECILER,77.351301
2,M2,6,HALIC,73.949640
3,M2,6,SISHANE,155.010811
4,M2,6,TAKSIM,240.980609
...,...,...,...,...
6835,T5-R,23,BALAT,24.085976
6836,T5-R,23,FENER,25.284440
6837,T5-R,23,CIBALI,30.405205
6838,T5-R,23,KUCUKPAZAR,39.270216


In [13]:
#cumulative sum of passenger_count shifted by 1 station
df_split['cum_passenger_count'] = df_split.groupby(['hour', 'line'])['in_passenger'].cumsum()
df_split['cum_passenger_count'] = df_split.groupby(['hour', 'line'])['cum_passenger_count'].shift(fill_value=0)
df_split

,line,hour,station,in_passenger,cum_passenger_count
0,M2,6,YENIKAPI,1198.250000,0.000000
1,M2,6,VEZNECILER,77.351301,1198.250000
2,M2,6,HALIC,73.949640,1275.601301
3,M2,6,SISHANE,155.010811,1349.550941
4,M2,6,TAKSIM,240.980609,1504.561752
...,...,...,...,...,...
6835,T5-R,23,BALAT,24.085976,331.864853
6836,T5-R,23,FENER,25.284440,355.950828
6837,T5-R,23,CIBALI,30.405205,381.235268
6838,T5-R,23,KUCUKPAZAR,39.270216,411.640473


In [14]:
def calculate_out_passenger_count(data, hour, line, station):
    data = data[(data["hour"] == hour) & (data["line"] == line)]
    station_data = data[data["station"] == station]
    passenger_count = station_data["in_passenger"].iloc[0]
    total_passenger_count = sum(data["in_passenger"])
    cumsum_passenger_count = station_data["cum_passenger_count"].iloc[0]
    out = passenger_count * cumsum_passenger_count / (total_passenger_count-passenger_count)
    return out


In [16]:
df_split['out_passenger'] = None


for index, row in df_split.iterrows():
    out = calculate_out_passenger_count(df_split, row['hour'], row['line'], row['station'])
    df_split.at[index, 'out_passenger'] = out

df_split['out_passenger'] = pd.to_numeric(df_split['out_passenger'], errors='coerce')
df_split["current_passenger"] = df_split["cum_passenger_count"] - df_split["out_passenger"] + df_split["in_passenger"]
df_split

,line,hour,station,in_passenger,cum_passenger_count,out_passenger,current_passenger
0,M2,6,YENIKAPI,1198.250000,0.000000,0.000000,1198.250000
1,M2,6,VEZNECILER,77.351301,1198.250000,33.078833,1242.522469
2,M2,6,HALIC,73.949640,1275.601301,33.624761,1315.926180
3,M2,6,SISHANE,155.010811,1349.550941,76.787977,1427.773775
4,M2,6,TAKSIM,240.980609,1504.561752,137.423102,1608.119259
...,...,...,...,...,...,...,...
6835,T5-R,23,BALAT,24.085976,331.864853,18.727333,337.223495
6836,T5-R,23,FENER,25.284440,355.950828,21.145353,360.089915
6837,T5-R,23,CIBALI,30.405205,381.235268,27.565720,384.074753
6838,T5-R,23,KUCUKPAZAR,39.270216,411.640473,39.270216,411.640473


In [ ]:
NUM_LINES = 5  # example number of metro lines
POPULATION_SIZE = 50
OPERATIONAL_COST = 1000
GENERATIONS = 100
FREQ_BOUNDS = (2, 10) 
STOP_TIME_BOUNDS = (20, 90)  # Seconds

w1, w2, w3, w4 = 1.0, 3.0, 1.5, 1.2

In [ ]:
def cost_function(frequency, stop_time, line_data, max_capacity, max_seat_capacity):
    op_cost = 0

    for date in line_data:
        for hour in line_data[date]:
            op_cost += frequency * OPERATIONAL_COST
            for station in line_data[date][hour]:

                wait_cost = station["in_passenger"] * frequency/2 + stop_time + station["cum_passenger_count"] * stop_time
                unserved_cost = (station["in_passenger"] - (max_capacity - station["cum_passenger_count"])) * unserved_cost 
                # station["cum_passenger_count"] is number of passengers when train reaches the station number of available places for ht new passengers is calculated as
                # max_capacity - station["cum_passenger_count"] and the remaining passengers are unserved
                density_cost = station["cum_passenger_count"] * density_cost 
                # because the cum_direction_1 is the number of passengers in the train harm of the density affects until the next station
                standing_cost = (station["cum_passenger_count"] - max_seat_capacity) * standing_cost

    return op_cost + wait_cost + unserved_cost + density_cost + standing_cost


In [ ]:
def initialize_population():
    population = []
    for _ in range(POPULATION_SIZE):
        frequencies = [random.randint(*FREQ_BOUNDS) for _ in range(NUM_LINES)]
        stop_times = [random.randint(*STOP_TIME_BOUNDS) for _ in range(NUM_LINES)]
        population.append((frequencies, stop_times))
    return population


In [ ]:
def select_population(population, fitnesses):
    sorted_pop = [x for _, x in sorted(zip(fitnesses, population))]
    return sorted_pop[:POPULATION_SIZE // 2]


In [ ]:
def crossover(parents):
    offspring = []
    while len(offspring) < POPULATION_SIZE:
        p1, p2 = random.sample(parents, 2)
        point = random.randint(1, NUM_LINES - 1)
        new_freq = p1[0][:point] + p2[0][point:]
        new_stop = p1[1][:point] + p2[1][point:]
        offspring.append((new_freq, new_stop))
    return offspring

In [ ]:
def mutate(individual, mutation_rate=0.1):
    for i in range(NUM_LINES):
        if random.random() < mutation_rate:
            individual[0][i] = random.randint(*FREQ_BOUNDS)
        if random.random() < mutation_rate:
            individual[1][i] = random.randint(*STOP_TIME_BOUNDS)
    return individual

In [ ]:
def run_genetic_algorithm(passenger_data):
    population = initialize_population()
    for generation in range(GENERATIONS):
        fitnesses = [cost_function(freqs, stops, passenger_data) for freqs, stops in population]
        best = min(fitnesses)
        print(f"Generation {generation} - Best Cost: {best}")
        selected = select_population(population, fitnesses)
        offspring = crossover(selected)
        population = [mutate(ind) for ind in offspring]
    return population[np.argmin(fitnesses)]


In [ ]:
mock_passenger_data = None  # Replace with real preprocessed data.

In [ ]:
best_solution = run_genetic_algorithm(mock_passenger_data)
print("Best Solution:", best_solution)

Generation 0 - Best Cost: 0
Generation 1 - Best Cost: 0
Generation 2 - Best Cost: 0
Generation 3 - Best Cost: 0
Generation 4 - Best Cost: 0
Generation 5 - Best Cost: 0
Generation 6 - Best Cost: 0
Generation 7 - Best Cost: 0
Generation 8 - Best Cost: 0
Generation 9 - Best Cost: 0
Generation 10 - Best Cost: 0
Generation 11 - Best Cost: 0
Generation 12 - Best Cost: 0
Generation 13 - Best Cost: 0
Generation 14 - Best Cost: 0
Generation 15 - Best Cost: 0
Generation 16 - Best Cost: 0
Generation 17 - Best Cost: 0
Generation 18 - Best Cost: 0
Generation 19 - Best Cost: 0
Generation 20 - Best Cost: 0
Generation 21 - Best Cost: 0
Generation 22 - Best Cost: 0
Generation 23 - Best Cost: 0
Generation 24 - Best Cost: 0
Generation 25 - Best Cost: 0
Generation 26 - Best Cost: 0
Generation 27 - Best Cost: 0
Generation 28 - Best Cost: 0
Generation 29 - Best Cost: 0
Generation 30 - Best Cost: 0
Generation 31 - Best Cost: 0
Generation 32 - Best Cost: 0
Generation 33 - Best Cost: 0
Generation 34 - Best Cos